In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
from DataFin import FMPClient

import os
import dotenv

dotenv.load_dotenv()

fmp_api_key = os.getenv('FMP_API_KEY')

s3_bucket = os.getenv('S3_BUCKET_NAME')
aws_access_key =  os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key =  os.getenv('AWS_SECRET_ACCESS_KEY')


fmp = FMPClient(fmp_api_key)

In [21]:
amzn_day_start = fmp.get_end_of_day_full('amzn')[0]['open']

In [ ]:
amzn = fmp.get_live_price('amzn')
price = amzn[0]['price']
pct_change = round((amzn[0]['change'] / amzn_day_start) * 100, 2)
print(price, '...', f'{pct_change}%')


180.246 ..... -2.47%
